In [10]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/parsing/constituency/test-augmented.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/parsing/constituency/train-augmented.txt

In [11]:
with open('train-augmented.txt') as fopen:
    train = fopen.read().split('\n')
    
with open('test-augmented.txt') as fopen:
    test = fopen.read().split('\n')
    
train = list(filter(None, train))
test = list(filter(None, test))

In [12]:
len(train), len(test)

(3989, 442)

In [13]:
train = [f'(TOP {s})' for s in train]
test = [f'(TOP {s})' for s in test]

In [14]:
train[:5]

['(TOP (S (PP (IN Menurut) (NP (PRP dia))) (, ,) (NP-SBJ (NN penjualan) (NN saham) (DT tersebut)) (VP (MD akan) (VP (VB berlaku) (ADJP (JJ efektif)) (SBAR (IN apabila) (S (NP-SBJ-1 (NN syarat-syarat) (SBAR (IN sebagaimana) (S (NP-SBJ-2 (-NONE- *)) (VP (VB termuat) (NP (-NONE- *-2)) (PP (IN dalam) (NP (NN perjanjian) (NN pengikatan) (NP (NN jual) (NN beli)) (NN saham))))))) (VP (MD telah) (VP (VB terpenuhi) (NP (-NONE- *-1)))))))) (. .)))',
 '(TOP (S-1 (NP-SBJ (NN Tokoh) (NN pasar)) (PRN (, ,) (SINV (VP (VB lanjut)  (S (-NONE- *T*-1))) (NP-SBJ (PRP dia)))) (VP (MD bagaimana) (VP (VB memerintah) (NP (NN keluarnya) (NP (NP (NP (NN dokumen) (NN indikator) (NN jimat)) (NNP AS)) (SBAR (IN yang) (S (NP-SBJ (-NONE- *)) (VP (MD akan) (VP (VB menunjukkan) (SBAR (-NONE- 0) (S (NP-SBJ (NP (NN pendirian) (NN jimat)) (NNP AS)) (VP (VP (VB mengelola)) (CC atau) (VP (ADVP (RB makin)) (VB lenyap)))))))))))))))',
 '(TOP (S-1 (NP-SBJ (NN Pelaku) (NN pasar)) (PRN (, ,) (SINV (VP (VB lanjut)  (S (-NONE- *T

In [15]:
with open('train.txt', 'w') as fopen:
    fopen.write('\n'.join(train))
    
with open('test.txt', 'w') as fopen:
    fopen.write('\n'.join(test))

In [16]:
from src.trees import *

In [17]:
with open('train.txt') as infile:
    treebanks = infile.read().split('\n')

In [18]:
trees = []
for treebank in treebanks:
    try:
        tokens = treebank.replace("(", " ( ").replace(")", " ) ").split()
        def helper(index):
            trees = []

            while index < len(tokens) and tokens[index] == "(":
                paren_count = 0
                while tokens[index] == "(":
                    index += 1
                    paren_count += 1

                label = tokens[index]
                index += 1

                if tokens[index] == "(":
                    children, index = helper(index)
                    trees.append(InternalTreebankNode(label, children))
                else:
                    word = tokens[index]
                    index += 1
                    trees.append(LeafTreebankNode(label, word))

                while paren_count > 0:
                    # print(tokens[index])
                    assert tokens[index] == ")"
                    index += 1
                    paren_count -= 1

            return trees, index
        ts, index = helper(0)
        trees.extend(ts)
    except:
        pass

In [19]:
len(trees)

3912

In [20]:
for i, tree in enumerate(trees):
    if tree.label in ("TOP", "ROOT"):
        assert len(tree.children) == 1
        trees[i] = tree.children[0]